In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, RFE
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import metrics
from sklearn.preprocessing import FunctionTransformer,Binarizer, Imputer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline, make_union, FeatureUnion
from sklearn.preprocessing import StandardScaler
%matplotlib inline

DATA CLEANING AND EXPLORATORY ANALYSIS NOTEBOOK
==

In [135]:
df = pd.read_csv('school_data_training.csv')
df.head()

,Name of School,"Elementary, Middle, or High School",Street Address,ZIP Code,Link,Healthy Schools Certified?,Safety Icon,Safety Score,Family Involvement Icon,Family Involvement Score,...,College Enrollment Rate %,College Enrollment (number of students),General Services Route,Freshman on Track Rate %,Community Area Number,Community Area Name,Ward,Police District,probation,Id
0,John Spry Elementary Community School,ES,2400 S Marshall Blvd,60623,http://schoolreports.cps.edu/SchoolProgressRep...,No,Strong,66.0,Average,59,...,NDA,809,39,NDA,30,SOUTH LAWNDALE,12,10,0,610184
1,Thomas A Edison Regional Gifted Center Element...,ES,4929 N Sawyer Ave,60625,http://schoolreports.cps.edu/SchoolProgressRep...,No,Very Strong,91.0,NDA,NDA,...,NDA,269,31,NDA,14,ALBANY PARK,39,17,0,609794
2,Milton Brunson Math & Science Specialty Elemen...,ES,932 N Central Ave,60651,http://schoolreports.cps.edu/SchoolProgressRep...,No,Weak,30.0,NDA,NDA,...,NDA,658,36,NDA,25,AUSTIN,29,15,1,609830
3,Emil G Hirsch Metropolitan High School,HS,7740 S Ingleside Ave,60619,http://schoolreports.cps.edu/SchoolProgressRep...,No,Very Weak,13.0,Average,46,...,45.1,458,46,69.5,69,GREATER GRAND CROSSING,8,6,1,609712
4,Lawndale Elementary Community Academy,ES,3500 W Douglas Blvd,60623,http://schoolreports.cps.edu/SchoolProgressRep...,No,NDA,NaN,NDA,NDA,...,NDA,508,37,NDA,29,NORTH LAWNDALE,24,10,1,610034


In [136]:
df = pd.concat([df,pd.get_dummies(df['Elementary, Middle, or High School'],drop_first=True,prefix = 'EMS')], axis = 1)
df.drop('Elementary, Middle, or High School',axis = 1, inplace = True)
#to get dummies and drop original column

In [137]:
df = pd.concat([df,pd.get_dummies(df['ZIP Code'],drop_first = True, prefix = 'ZIP')], axis = 1)
df.drop('ZIP Code', axis = 1, inplace = True)

In [138]:
df['Healthy Schools Certified?'] = df['Healthy Schools Certified?'].apply(lambda x: 1 if x == 'Yes' else 0)
#to make binary (0,1)

In [157]:
df.loc[df['Safety Icon '] == 'NDA', 'Safety Icon '] = 'Average'
#consider using this feature with caution due to NDA values (34)

In [158]:
df['Safety Icon '].value_counts()

Average        173
Weak           117
Strong          74
Very Strong     32
Very Weak       18
Name: Safety Icon , dtype: int64

In [190]:
df = pd.concat([df,pd.get_dummies(df['Safety Icon '],drop_first = True, prefix = 'SafetyIcon')], axis = 1)
df.drop('Safety Icon ', axis = 1, inplace = True)

In [159]:
df['Safety Score'].describe()

count    380.000000
mean      49.023684
std       19.891977
min        1.000000
25%       34.000000
50%       46.000000
75%       61.000000
max       99.000000
Name: Safety Score, dtype: float64

In [161]:
df['Safety Score'].fillna(49.0, inplace = True)

In [166]:
df.loc[df['Family Involvement Icon'] == 'NDA', 'Family Involvement Icon'] = 'Average'
#consider using this feature with caution due to NDA values (196)

In [191]:
df = pd.concat([df,pd.get_dummies(df['Family Involvement Icon'],drop_first = True, prefix = 'FamilyInvolvementIcon')], axis = 1)
df.drop('Family Involvement Icon', axis = 1, inplace = True)

In [176]:
df.loc[df['Family Involvement Score'] == 'NDA','Family Involvement Score'] = 50
#consider using this feature with caution due to NDA values (196)

In [184]:
df['Family Involvement Score'] = df['Family Involvement Score'].apply(int)

In [189]:
df.loc[df['Environment Icon '] == 'NDA','Environment Icon '] = 'Average'
#consider using this feature with caution due to NDA values (34)

In [192]:
df = pd.concat([df,pd.get_dummies(df['Environment Icon '],drop_first = True, prefix = 'EnvironmentIcon')], axis = 1)
df.drop('Environment Icon ', axis = 1, inplace = True)

In [197]:
df['Environment Score'].fillna(47.0, inplace = True)
#consider using this feature with caution due to NDA values (34)

In [200]:
df.loc[df['Instruction Icon '] == 'NDA','Instruction Icon '] = 'Average'
#consider using this feature with caution due to NDA values (34)

In [202]:
df = pd.concat([df,pd.get_dummies(df['Instruction Icon '],drop_first = True, prefix = 'InstructiontIcon')], axis = 1)
df.drop('Instruction Icon ', axis = 1, inplace = True)

In [205]:
df['Instruction Score'].fillna(48.0, inplace = True)
#consider using this feature with caution due to NDA values (34)

In [208]:
df.loc[df['Leaders Icon '] == 'NDA', 'Leaders Icon '] = 'Average'
#consider using this feature with caution due to NDA values (196)

In [209]:
df = pd.concat([df,pd.get_dummies(df['Leaders Icon '],drop_first = True, prefix = 'LeaderIcon')], axis = 1)
df.drop('Leaders Icon ', axis = 1, inplace = True)

In [216]:
df.loc[df['Leaders Score '] == 'NDA','Leaders Score '] = 49
#consider using this feature with caution due to NDA values (196)

In [217]:
df['Leaders Score '] = df['Leaders Score '].apply(int)

In [221]:
df.loc[df['Teachers Icon '] == 'NDA', 'Teachers Icon '] = 'Average'
#consider using this feature with caution due to NDA values (197)

In [222]:
df = pd.concat([df,pd.get_dummies(df['Teachers Icon '],drop_first = True, prefix = 'TeachersIcon')], axis = 1)
df.drop('Teachers Icon ', axis = 1, inplace = True)

In [225]:
df.loc[df['Teachers Score'] == 'NDA','Teachers Score'] = 48.0
#consider using this feature with caution due to NDA values (197)

In [227]:
df['Teachers Score'] = df['Teachers Score'].apply(int)

In [232]:
df.loc[df['Parent Engagement Icon '] == 'NDA', 'Parent Engagement Icon '] = 'Average'
#consider using this feature with caution due to NDA values (99)

In [242]:
df = pd.concat([df,pd.get_dummies(df['Parent Engagement Icon '],drop_first = True, prefix = 'ParentEngIcon')], axis = 1)
df.drop('Parent Engagement Icon ', axis = 1, inplace = True)

In [237]:
df.loc[df['Parent Engagement Score'] == 'NDA', 'Parent Engagement Score'] = 50.0

In [238]:
df['Parent Engagement Score'] = df['Parent Engagement Score'].apply(int)
#consider using this feature with caution due to NDA values (99)

In [241]:
df.loc[df['Parent Environment Icon'] == 'NDA', 'Parent Environment Icon'] = 'Average'

In [243]:
df = pd.concat([df,pd.get_dummies(df['Parent Environment Icon'],drop_first = True, prefix = 'ParentEnvIcon')], axis = 1)
df.drop('Parent Environment Icon', axis = 1, inplace = True)

In [246]:
df.loc[df['Parent Environment Score'] == 'NDA', 'Parent Environment Score'] = 50.0

In [247]:
df['Parent Environment Score'] = df['Parent Environment Score'].apply(int)

In [270]:
def percentage(series):
    series = series.replace('%','')
    return float(series)
#create function to convert percentage columns to float

In [271]:
df['Average Student Attendance'] = df['Average Student Attendance'].apply(percentage)

In [282]:
df['Rate of Misconducts (per 100 students) '].isnull().sum()

0

In [284]:
df['Average Teacher Attendance'] = df['Average Teacher Attendance'].apply(percentage)

In [288]:
df['Individualized Education Program Compliance Rate '] = df['Individualized Education Program Compliance Rate '].apply(percentage)

In [295]:
df.loc[df['Pk-2 Literacy %'] == 'NDA', 'Pk-2 Literacy %'] = 64.0

In [297]:
df['Pk-2 Literacy %'] = df['Pk-2 Literacy %'].apply(float)

In [302]:
df.loc[df['Pk-2 Math %'] == 'NDA', 'Pk-2 Math %'] = 44.0

In [303]:
df['Pk-2 Math %'] = df['Pk-2 Math %'].apply(float)

In [322]:
df.loc[df['Gr3-5 Grade Level Math %'] == 'NDA', 'Gr3-5 Grade Level Math %'] = 38.0

In [323]:
df['Gr3-5 Grade Level Math %'] = df['Gr3-5 Grade Level Math %'].apply(float)

In [328]:
def mathlit(df, column):
    mean = df.loc[df[column] != 'NDA', column].astype(float).mean()
    df.loc[df[column] == 'NDA', column] = mean
    df[column] = df[column].apply(float)
    return df[column]
#make a function to apply for these types of columns, (has NDAs, string, no %)

In [347]:
lis = ['Gr3-5 Keep Pace Math %','Gr3-5 Keep Pace Read %','Gr6-8 Grade Level Math %','Gr6-8 Grade Level Read %',
      'Gr6-8 Keep Pace Math%','Gr6-8 Keep Pace Read %','Gr-8 Explore Math %','Gr-8 Explore Read %']
for li in lis:
    df[li] = mathlit(df,li)

In [353]:
df['ISAT Value Add Math'].fillna(df['ISAT Value Add Math'].mean(), inplace = True)

In [359]:
df['ISAT Value Add Read'].fillna(df['ISAT Value Add Read'].mean(), inplace = True)

In [363]:
df['ISAT Exceeding Math %'].fillna(df['ISAT Exceeding Math %'].mean(), inplace = True)

In [367]:
df['ISAT Exceeding Reading % '].fillna(df['ISAT Exceeding Reading % '].mean(), inplace = True)
#60 Nans

In [373]:
df.loc[df['ISAT Value Add Color Math'] == 'NDA', 'ISAT Value Add Color Math'] = 'Yellow'

In [374]:
df = pd.concat([df,pd.get_dummies(df['ISAT Value Add Color Math'],drop_first = True, prefix = 'ISATVAColorMath')], axis = 1)
df.drop('ISAT Value Add Color Math', axis = 1, inplace = True)

In [376]:
df.loc[df['ISAT Value Add Color Read'] == 'NDA', 'ISAT Value Add Color Read'] = 'Yellow'
#66 NDA

In [377]:
df = pd.concat([df,pd.get_dummies(df['ISAT Value Add Color Read'],drop_first = True, prefix = 'ISATVAColorRead')], axis = 1)
df.drop('ISAT Value Add Color Read', axis = 1, inplace = True)

In [403]:
liz = ['Students Taking  Algebra %','Students Passing  Algebra %','9th Grade EXPLORE (2009) ','9th Grade EXPLORE (2010) ',
      '10th Grade PLAN (2009) ','10th Grade PLAN (2010) ','Net Change EXPLORE and PLAN','11th Grade Average ACT (2011) ',
      'Net Change PLAN and ACT','College Eligibility %','Graduation Rate %','College Enrollment Rate %',
      'Freshman on Track Rate %']
for li in liz:
    df[li] = mathlit(df,li)
#all these have at least 300 NDAs

In [409]:
df = pd.concat([df,pd.get_dummies(df['Community Area Number'],drop_first = True, prefix = 'CommAreaNumber')], axis = 1)
df.drop('Community Area Number', axis = 1, inplace = True)

In [ ]:
df = pd.concat([df,pd.get_dummies(df['Community Area Number'],drop_first = True, prefix = 'CommAreaNumber')], axis = 1)
df.drop('Community Area Number', axis = 1, inplace = True)

In [411]:
df_areaname = df['Community Area Name']
#for reference purposes

In [412]:
df.drop('Community Area Name', axis = 1, inplace = True)

In [414]:
df = pd.concat([df,pd.get_dummies(df['Ward'],drop_first = True, prefix = 'Ward')], axis = 1)
df.drop('Ward', axis = 1, inplace = True)

In [416]:
df = pd.concat([df,pd.get_dummies(df['Police District'],drop_first = True, prefix = 'Police')], axis = 1)
df.drop('Police District', axis = 1, inplace = True)

In [417]:
df.head()

,Name of School,Street Address,Link,Healthy Schools Certified?,Safety Score,Family Involvement Score,Environment Score,Instruction Score,Leaders Score,Teachers Score,...,Police_14,Police_15,Police_16,Police_17,Police_18,Police_19,Police_20,Police_22,Police_24,Police_25
0,John Spry Elementary Community School,2400 S Marshall Blvd,http://schoolreports.cps.edu/SchoolProgressRep...,0,66.0,59,70.0,67.0,52,43,...,0,0,0,0,0,0,0,0,0,0
1,Thomas A Edison Regional Gifted Center Element...,4929 N Sawyer Ave,http://schoolreports.cps.edu/SchoolProgressRep...,0,91.0,50,64.0,56.0,49,48,...,0,0,0,1,0,0,0,0,0,0
2,Milton Brunson Math & Science Specialty Elemen...,932 N Central Ave,http://schoolreports.cps.edu/SchoolProgressRep...,0,30.0,50,30.0,45.0,49,48,...,0,1,0,0,0,0,0,0,0,0
3,Emil G Hirsch Metropolitan High School,7740 S Ingleside Ave,http://schoolreports.cps.edu/SchoolProgressRep...,0,13.0,46,28.0,28.0,52,50,...,0,0,0,0,0,0,0,0,0,0
4,Lawndale Elementary Community Academy,3500 W Douglas Blvd,http://schoolreports.cps.edu/SchoolProgressRep...,0,49.0,50,47.0,48.0,49,48,...,0,0,0,0,0,0,0,0,0,0


In [419]:
df1 = df
#create another dataframe for dropping name, address, link (might try making features for them later)

In [420]:
y = df['probation'].values

In [422]:
X = df1.drop(['Name of School','Street Address','Link '], axis = 1)

In [423]:
X.head()
#df with all the available features

,Healthy Schools Certified?,Safety Score,Family Involvement Score,Environment Score,Instruction Score,Leaders Score,Teachers Score,Parent Engagement Score,Parent Environment Score,Average Student Attendance,...,Police_14,Police_15,Police_16,Police_17,Police_18,Police_19,Police_20,Police_22,Police_24,Police_25
0,0,66.0,59,70.0,67.0,52,43,46,48,96.2,...,0,0,0,0,0,0,0,0,0,0
1,0,91.0,50,64.0,56.0,49,48,55,57,96.6,...,0,0,0,1,0,0,0,0,0,0
2,0,30.0,50,30.0,45.0,49,48,47,55,91.3,...,0,1,0,0,0,0,0,0,0,0
3,0,13.0,46,28.0,28.0,52,50,50,50,84.8,...,0,0,0,0,0,0,0,0,0,0
4,0,49.0,50,47.0,48.0,49,48,50,50,91.0,...,0,0,0,0,0,0,0,0,0,0


In [424]:
X.to_csv('p5classification.csv')
#create clean CSV for modelling notebook (not sure if this is the correct workflow method in making pipelines)

In [426]:
yexport = pd.DataFrame(y)

In [428]:
yexport.to_csv('p5cloutput.csv')
#export target for modelling notebook

In [44]:
for col in df.columns:
    print col
#for easier reference

Name of School
Street Address
Link 
Healthy Schools Certified?
Safety Icon 
Safety Score
Family Involvement Icon
Family Involvement Score
Environment Icon 
Environment Score
Instruction Icon 
Instruction Score
Leaders Icon 
Leaders Score 
Teachers Icon 
Teachers Score
Parent Engagement Icon 
Parent Engagement Score
Parent Environment Icon
Parent Environment Score
Average Student Attendance
Rate of Misconducts (per 100 students) 
Average Teacher Attendance
Individualized Education Program Compliance Rate 
Pk-2 Literacy %
Pk-2 Math %
Gr3-5 Grade Level Math %
Gr3-5 Grade Level Read % 
Gr3-5 Keep Pace Read %
Gr3-5 Keep Pace Math %
Gr6-8 Grade Level Math %
Gr6-8 Grade Level Read %
Gr6-8 Keep Pace Math%
Gr6-8 Keep Pace Read %
Gr-8 Explore Math %
Gr-8 Explore Read %
ISAT Exceeding Math %
ISAT Exceeding Reading % 
ISAT Value Add Math
ISAT Value Add Read
ISAT Value Add Color Math
ISAT Value Add Color Read
Students Taking  Algebra %
Students Passing  Algebra %
9th Grade EXPLORE (2009) 
9th Grade

Notes on treatment of each column
--

Notes:
Name of School - try to get keywords?
EMS - get dummies
Street Address - keywords?
ZIP - dummies
Link - keywords?
Healthy Schools Certified - get dummies
Safety Icon - get dummies
Safety Score - use, fill na
Family Involvement Icon - get dummies
Family Involvement Score - use, fill na
Environment Icon - get dummies
Environment Score - use, fill na
Instruction Icon - get dummies
Instruction Score - use, fill na
Leaders Icon - get dummies
Leaders Score - use, fill na
Teachers Icon - get dummies
Teachers Score - use, fill na
Parent Engagement Icon - get dummies
Parent Engagement Score - use, fill na
Parent Environment Icon - get dummies
Parent Environment Score - use, fill na
Average Student Attendance - use, has %
Rate of Misconducts per 100 students - use, fill na
Average Teacher Attendance - use, has %
Individualized Education Program Compliance Rate - use, has %
Pk-2 Literacy - use, has lot of NDA
Pk-2 Math - use, has lot of NDA
Gr3-5 Grade Level Math - use, has lot of NDA
Gr3-5 Grade Level Read - use, has lot of NDA
Gr3-5 Keep Pace Read - use, has lot of NDA
Gr3-5 Keep Pace Math - use, has lot of NDA
Gr6-8 Grade Level Math % - use, has lot of NDA
Gr6-8 Grade Level Read - use, has lot of NDA
Gr6-8 Keep Pace Math - use, has lot of NDA
Gr6-8 Keep Pace Read - use, has lot of NDA
Gr-8 Explore Math - use, has lot of NDA
Gr-8 Explore Read - use, has lot of NDA
ISAT Exceeding Math - use, no NDA
ISAT Exceeding Read - use, no NDA
ISAT Add Value Math - use, no NDA
ISAT Add Value Read - use, no NDA
ISAT Add Value Color Math - get dummies
ISAT Add Value Color Read - get dummies
Students Taking  Algebra % - use, has lot of NDA
Students Passing Algebra % - use, has lot of NDA
9th Grade Explore (2009) - use, has lot of NDA
9th Grade Explore (2010) - use, has lot of NDA
10th Grade PLAN (2009) - use, has lot of NDA
10th Grade PLAN (2010) - use, has lot of NDA
Net Change EXPLORE and PLAN - use, has lot of NDA
11th Grade Average ACT (2011) - use, has lot of NDA
Net Change PLAN and ACT - use, has lot of NDA
College Eligibility % - use, has lot of NDA, has %
Graduation Rate - use, has lot of NDA
College Enrollment Rate - use, has lot of NDA
College Enrollment (number of students) - use
General Services Route - use (asbestos surveillance???)
Freshman on Track Rate % - use, lot of NDA
Community Area Name - create dummies
Ward - create dummies
Police - District - create dummies
probation - TARGET



Notes from Friday

Model Generation:
Question
Data Acquisition (CSV, database, scrape)
Data Exploration (Types, columns, shape, data generation process, missing data)
Operationalize Y
Modeling (Model type, Iteration (Feature Selection: KBest, Intuition, correlation, RFE, Regularization, Experiment),
Transformation)
Goodness of Fit - Regression(R^2, MSE, plot predict versus actual), no overfitting (always cross validate)
- Classification (confusion matrix, Classification Report, mean accuracy)
Linear regression
knn, logistic
Hyperparameters

Pipeline:
Raw Data
Data Transformer(pandas, functions, skpipeline)
Model
Predictions
